<a href="https://colab.research.google.com/github/colinmcnamara/Digital_Deming/blob/main/Digital_Deming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up your local environment 

On intial run of "import colab_env" it will create vars.env in the root of your Google Drive

COLAB_ENV = Active

export OPENAI_API_KEY = "your_key"

export SERPAPI_API_KEY = "your_key"


Install my local packages

In [86]:
pip install OpenAI langchain pydantic faiss-gpu colab-env unstructured chromadb tiktoken pdf2image faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Yo, let's import our local env so we don't have to show the world our keys

In [87]:
import colab_env

Let's import our python modules

In [88]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import magic
import os
import nltk
import textwrap



Let's set the temperature of our AI models

In [89]:
llm = OpenAI(temperature=0)
chat = ChatOpenAI(temperature=0)

Let's specify the directory and file type for our Demming source docs

In [90]:
loader = DirectoryLoader('/content/gdrive/MyDrive/Projects/Digital_Deming/src_content', glob='**/*.pdf')

Load our documents


In [91]:
documents = loader.load()

Check how many documents are loaded

In [92]:
len (documents)

1

Let's split our document up into chunks

In [93]:
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    separators=["\n\n", "\n", " ", ""]
)

Let' create a variable named texts, and split these documents into it

In [94]:
texts = text_splitter.split_documents(documents)

Let's tell it that we want it to use OpenAI embeddings, and tell it what env variable the key is held

In [95]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

Let's create a variable named docsearch, and pull from Chroma vectorstore including texts, and the embeddings)

In [96]:
docsearch = Chroma.from_documents(texts, embeddings)

define qa chain, which pulls from VectorDBQA, and use the LLM chain named OpenAI. We specify that it "Stuff"'s all the data in. We could use "map_reduce" instead to optimize for performance and scale. Let's tell it to use the vectorstore we defined earlier named docsearch

note, i'm trying RetrievalQA because VectorDBQA is depricated

In [97]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
query = "Describe in depth, what is the foundation of quality. Provide three examples using bullet points"
result = qa({"query": query})

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:206: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [98]:
result['result']

'\n\nThe foundation of quality is the implementation of a set of quality control processes and systems that ensure products and services meet customer expectations. Examples include:\n\n• Establishing quality management systems that define how quality is monitored and maintained\n• Developing quality assurance processes that ensure products and services meet customer expectations\n• Implementing quality control processes that identify and address potential quality issues before they become a problem'

In [99]:
result['source_documents']

[Document(page_content='Major Events in the Development of QC\n\nCourse. (July–Dec. 1957, JUSE)\n\n\uf09f The Chugoku Quality Control Research Group was established. (JSA Hiroshima) \uf09f The Design of Experiment Seminar was\n\nheld. (JSA)\n\n\uf09f The European Organization for Quality\n\nControl (EOQC) was founded.\n\n\uf09f A. V. Feigenbaum published his thesis titled “Total Quality Control” in the Harvard Business Review.\n\n1957\n\n\uf09f He appeared as a lecturer on NHK’s radio\n\n\uf09f The first Spring QC Conference was\n\nprograms. (–62, NHK Radio 2)\n\nheld. (Osaka, May, JUSE)\n\n\uf09f His article “Points in Preparing In-house Standards” (published in the Hinshitsu Kanri (Statistical Quality Control) [140]) won the Standardization Literature Award.\n\n\uf09f He started playing golf. \uf09f He published Introduction to Sampling\n\n\uf09f The Operations Research Society of\n\nJapan was founded. (June)\n\n\uf09f The QC Seminar for Executives was held\n\nin Karuizawa. (July, JU

In [100]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "what is the most important concept in TQM"
#query =  "WHo won the deming prize?"
#query = "Is TQM like lean?"
#query = "How are TQM and Lean different?"
query = "What is lean?"

result = qa({"query": query})

In [101]:
result['result']

" I don't know."

In [102]:
result['source_documents']

[Document(page_content='product liability.\n\n1976 March 31\n\nApril 1\n\n\uf09f Ishikawa retired from the University of\n\n\uf09f The Iranian QC Research Team visited\n\nTokyo.\n\nJapan. (APO)\n\n\uf09f He became a professor of the Faculty of Engineering, Science University of Tokyo.\n\n\uf09f The 1st East Asia QC Circle International\n\nExchange Meeting was held. (Apr., Seoul)\n\nMay 18\n\n\uf09f He became a professor emeritus of the\n\n\uf09f The TQC Promotion Seminar for\n\nUniversity of Tokyo.\n\n\uf09f He was awarded with the Asada Medal\n\nfrom the Iron and Steel Institute of Japan.\n\n\uf09f He became the head of the 2nd\n\nInternationally-Operating Company QC Research Team dispatched to Central and South America. (JUSE)\n\n\uf09f He became a member of PASC. \uf09f He became an honorary member of the\n\nPhilippine Society for Quality.\n\n\uf09f He became the director of JUSE Press. \uf09f He was awarded with the Fuel Society of\n\nPractioners was held. [79–, TQC Seminar for Fac

# The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)

## I. Introduction
- Definition of Total Quality Management (TQM)
- Brief introduction of the significance of TQM in the modern business world
- Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
- Thesis Statement

In [103]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "what is Total Quality Management (TQM)"
result = qa({"query": query})

In [104]:
result['result']

' Total Quality Management (TQM) is a management approach that focuses on continuous improvement of quality in order to meet customer requirements and improve the overall performance of an organization. It is a system for ensuring that all processes and products meet the highest standards of quality, from design to delivery.'

In [105]:
result['source_documents']

[Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published in April, 1988, JSA)\n\n\uf09f The 25th QCS: Small-group activities\n\nand QC.\n\n1978\n\nApril 1\n\n\uf09f He became the President of Musashi\n\n\uf09f The Construction Industry TQC\n\nInstitute of Technology.\n\nRound-table Conference was held. (JSA)\n\n\uf09f He became the director of Goto-Ikueikai. \uf09f He became the director of the Tokyu Foundation for Foreign Students.\n\n\uf09f He became the chairper

In [106]:
# Create a list to stuff section results into 
introduction = []

#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is a management approach that focuses on continuous improvement of quality in order to meet customer requirements and improve the overall performance of an organization. It is a system for ensuring that all processes and products meet the highest standards of quality, from design to delivery.', 'source_documents': [Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published 

In [107]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "Write a detailed introduction of the significance of TQM in the modern business world"
result = qa({"query": query})

In [108]:
result['result']

'\n\nTotal Quality Management (TQM) is an approach to business management that focuses on improving the effectiveness and efficiency of operations, as well as consistently delivering the highest quality products and services to customers. It is based on the principles of continuous improvement, customer satisfaction, and employee engagement. TQM seeks to eliminate waste in all forms and to use resources in the most efficient way. It also encourages teams to identify opportunities for improvement, to analyze problems, and to develop creative solutions. In this way, TQM helps businesses to reduce costs, increase profitability, and improve customer satisfaction. It also helps to foster a culture of trust, collaboration, and accountability among employees. TQM is a powerful tool for businesses of all sizes, from small businesses to large corporations, and it is increasingly being adopted by organizations around the world.'

In [109]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is a management approach that focuses on continuous improvement of quality in order to meet customer requirements and improve the overall performance of an organization. It is a system for ensuring that all processes and products meet the highest standards of quality, from design to delivery.', 'source_documents': [Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published 

In [110]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
result = qa({"query": query})

In [111]:
result['result']

' Kaoru Ishikawa was a lecturer at the University of Tokyo in the Department of Fuel Engineering. He was also a member of the ISO/TC 102 Japan Committee and served as the chair of the Sampling Subcommittee. In 1949, he became an instructor of the QC Basic Course for the Japanese Union of Scientists and Engineers. He gave the first lecture in the QC Basic Course and introduced Rice’s Control Chart in July of that year. He was also the vice-chair of the Editorial Committee of Hinshitsu Kanri (Statistical Quality Control) from 1950-52. In 1956, he became the chief of ASQC Japan Branch.'

In [112]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is a management approach that focuses on continuous improvement of quality in order to meet customer requirements and improve the overall performance of an organization. It is a system for ensuring that all processes and products meet the highest standards of quality, from design to delivery.', 'source_documents': [Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published 

In [113]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
#query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
query = " Write a Thesis Statement regarding TQM, Kaoru Ishikawa, Japanese Union of Scientists and engineers."
result = qa({"query": query})

In [114]:
result['result']

" Kaoru Ishikawa's contributions to TQM, such as his research on coal briquettes, membership in the Japanese Association of University Professors, and involvement in the International Organization for Standardization, helped to shape the development of TQM in Japan as part of the Japanese Union of Scientists and Engineers."

In [115]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is a management approach that focuses on continuous improvement of quality in order to meet customer requirements and improve the overall performance of an organization. It is a system for ensuring that all processes and products meet the highest standards of quality, from design to delivery.', 'source_documents': [Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published 

Let's see if we can output this to markdown

In [116]:
concatenated_data = [str(item) for item in introduction]
for entry in concatenated_data:
    print("Processing entry: ")
    messages = [
        SystemMessage(content="You are an AI that takes dictionary data and translates it into well formatted markdown. You create sections and subsections to communicate as much information as possible"),
        HumanMessage(content=entry)
    ]
    result = chat(messages)

    print("Result: ", result)

Processing entry: 
Result:  content='# Total Quality Management (TQM)\n\nTotal Quality Management (TQM) is a management approach that focuses on continuous improvement of quality in order to meet customer requirements and improve the overall performance of an organization. It is a system for ensuring that all processes and products meet the highest standards of quality, from design to delivery.\n\n## Source Documents\n\nThe information on Total Quality Management (TQM) was sourced from the following documents:\n\n- Medal. (ASQC)\n- The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n- The Japan Prefabricated Construction Suppliers and Manufacturers Association held TQC seminars.\n- Noboru Takagi was elected as the 22nd president of the IEC.\n- The 4th ICPR Conference (IE international conference) was held.\n- The 24th QCS: Quality design and process design written and edited by Ishikawa was published. (The second edition was published in April, 19

In [117]:

  messages = [
      SystemMessage(content="You are an assistant helping me research a paper"),
      HumanMessage(content="return an outline for the book - The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE) ")
  ]
  result = chat(messages)

  print("Result: ", result)

Result:  content="I. Introduction\n- Brief overview of Total Quality Management (TQM)\n- Importance of studying the history and evolution of TQM\n- Thesis statement\n\nII. The Origins of TQM\n- Early quality control practices in manufacturing\n- The role of W. Edwards Deming in the development of TQM\n- The impact of World War II on TQM\n\nIII. Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)\n- Ishikawa's background and contributions to TQM\n- The formation and goals of JUSE\n- The development of Ishikawa's quality control tools\n\nIV. The Spread of TQM\n- The influence of JUSE on TQM in Japan\n- The adoption of TQM by American companies\n- The impact of TQM on global business practices\n\nV. Criticisms and Challenges of TQM\n- Criticisms of TQM's effectiveness\n- Challenges in implementing TQM in organizations\n- The future of TQM\n\nVI. Conclusion\n- Summary of key points\n- Implications for future research and practice." additional_kwargs={} example=False


In [118]:
#from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Here is the structure of the paper:

The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
I. Introduction
Definition of Total Quality Management (TQM)
Brief introduction of the significance of TQM in the modern business world
Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
Thesis Statement
III. The Birth and Evolution of TQM
Introduction to the concept of TQM
Overview of the principles and practices of TQM
The role of TQM in post-World War II Japan's industrial comeback
Adoption and evolution of TQM in the United States and around the world
IV. The Role of Kaoru Ishikawa in the Development of TQM
Introduction to Ishikawa: his background and career
Ishikawa's contribution to the development of Quality Circles
The development and importance of the Ishikawa (cause-and-effect) Diagram
Ishikawa's philosophical contribution to TQM: the 'next process is your customer'
V. The Role of the Japanese Union of Scientists and Engineers (JUSE)
Background of JUSE and its role in the development of quality management in Japan
Introduction of the Deming Prize by JUSE and its significance
JUSE's role in promoting TQM and its principles in Japan and globally
VI. The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices
Analysis of the role TQM plays in today's business and management practices
The legacy of Ishikawa's methods and tools in current quality management
Influence of JUSE's principles in contemporary quality initiatives and awards
VII. Conclusion
Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management
Restatement of Thesis
Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape

{context}

Question: {question}
Answer as if you are a quality management consultant. Use specific dates and references. Use bullet points and tabels where appropriate. Reference source documents:

"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)
query = "Who won the Deming prize?"
qa.run(query)

'The first winners of the Deming Prize in 1980 were Motosaburo Masuyama, Showa Denko, Tanabe Pharmaceuticals, Fuji Iron & Steel, and Yawata Iron & Steel. \n\nMotosaburo Masuyama won the Deming Prize for Individuals and the other four companies won the Deming Application Prize. The Deming Prize was established by the Japanese Union of Scientists and Engineers (JUSE) in November 1980.'

In [119]:
query = "when was the last deming prize awarded"
qa.run(query)

'The last Deming Prize was awarded in November 2020. The Deming Prize Committee of the Japanese Union of Scientists and Engineers (JUSE) announced the winners of the 65th Deming Prize for Individuals and the 32nd Deming Application Prize. The winners of the 65th Deming Prize for Individuals were Dr. Tatsuo Yamamoto, President of the Japan Institute for Social and Economic Affairs, and Prof. Masayuki Yui, Professor Emeritus at the Graduate School of Economics, University of Tokyo. The 32nd Deming Application Prize was awarded to Tokio Marine & Nichido Fire Insurance Co., Ltd.'

In [142]:
query = "who invented QC circle?"
qa.run(query)

'Ishikawa is widely credited with inventing Quality Circles. His work in this area began in the early 1960s while he was teaching at the University of Tokyo. His efforts to promote Quality Circles were supported by the Japanese Union of Scientists and Engineers (JUSE) and led to the establishment of the first Quality Circle Conference in 1971 in Taiwan. Ishikawa\'s Quality Circles were further popularized by JUSE in the early 1970s, and in 1973, the 15th Quality Control Seminar was held with the theme of "QC as a tool to overcome the economic depression." This seminar was a major milestone in promoting the use of Quality Circles and solidified Ishikawa\'s role as the inventor of Quality Circles.'

In [121]:
query = "when was the QC Circle Seminar for Top Management held"
qa.run(query)

'The QC Circle Seminar for Top Management was held from 1973 to 1971, under the chairmanship of Professor Kaoru Ishikawa. The seminar was organized by JUSE (Japanese Union of Scientists and Engineers). During this time, the 43rd QCS (Quality Control Symposium) was held, discussing "New Technology Development/New Product Planning and Quality Control" and the 44th QCS discussed "10 Distinctive Features of Japanese Company-Wide Quality Control". In addition, the 25th Anniversary Party for QC Circles was held in April, with the publication of the 25-year History of QC Circles and a national survey on the spread of QC circle activities.'

# The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)

## I. Introduction
* Definition of Total Quality Management (TQM)
* Brief introduction of the significance of TQM in the modern business world
* Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
* Thesis Statement

In [138]:
query = "write a Definition of Total Quality Management (TQM)"
qa.run(query)

'Total Quality Management (TQM) is an approach to organizational management that emphasizes continuous improvement, customer satisfaction, and the involvement of all employees in the quality process. TQM was first developed in Japan in the 1950s and 1960s by Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE). Specifically, Ishikawa introduced the concept of Quality Circles, the Ishikawa (cause-and-effect) Diagram, and the principle of “the next process is your customer”. JUSE also established the Deming Prize for quality management in 1951, and promoted TQM principles in Japan and around the world. Today, TQM is widely used in the business world and is accepted as a key factor in the success of organizations. It is an essential element of modern quality management systems, such as ISO 9000 and Six Sigma.'

In [139]:
query = "write a Brief introduction of the significance of TQM in the modern business world"
qa.run(query)

"Total Quality Management (TQM) is a comprehensive approach to quality management that focuses on improving the efficiency and effectiveness of business processes. TQM has become a popular business strategy since its development in the 1950s and has been adopted by organizations around the world. In recent years, TQM has become increasingly important in the modern business world, as organizations strive to remain competitive in an ever-changing global market. \n\nTQM focuses on continually improving the quality of products and services by focusing on the customer, preventing defects, and improving processes. Its principles are applicable to all types of organizations, including manufacturing, healthcare, service, and government. \n\nThe development of TQM can be traced back to the 1950s, when Dr. W. Edwards Deming proposed a new system of quality management. Deming's work was further advanced by Kaoru Ishikawa, who developed the Ishikawa diagram, and the Japanese Union of Scientists an

In [140]:
query = "write an Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
qa.run(query)

'\nIntroduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)\n\nKaoru Ishikawa was born in 1915 in Tokyo, Japan, and was a highly influential figure in the field of quality management. He was a professor of engineering at the University of Tokyo, in the 4th Fuel Engineering Laboratory (Applied Combustible Science). After World War II, he played a major role in the development of Total Quality Management (TQM) in Japan and around the world. \n\nThe Japanese Union of Scientists and Engineers (JUSE) was founded in 1949 as a non-profit organization dedicated to the promotion of quality management in Japan and abroad. JUSE was instrumental in the development and promotion of TQM, and was the sponsor of the Deming Prize, which is awarded annually for excellence in quality management. \n\nThe Contributions of Kaoru Ishikawa and JUSE to Quality Management: \n\n• Kaoru Ishikawa was a key figure in the development of TQM, and was responsible for introducing several

In [141]:
query = "write an Thesis statement "
qa.run(query)

'Thesis Statement: TQM, as developed by Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), has had a profound impact on contemporary business practices in terms of quality management. By introducing the concepts of quality circles, the Ishikawa (cause-and-effect) Diagram, and the ‘next process is your customer’ philosophy, Ishikawa and JUSE have revolutionized quality management since the early 1930s. The establishment of the Deming Prize by JUSE in 1945, and the continued influence of its principles in contemporary quality initiatives and awards, are testament to its enduring legacy.'

## II. Early History of Quality Management
- Brief description of quality management before TQM
- Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance
- The rise of statistical process control and its significance

In [122]:
query = "write  Early History of Quality Management Brief description of quality management before TQM"
qa.run(query)

'Early History of Quality Management\n\nBefore TQM, quality management saw its beginnings in 1931 with the work of Dr. Walter Shewhart. His book, Economic Control of Quality of Manufactured Product, introduced the concept of quality control as a systematic approach to dealing with defects. This was followed by Yasushi Ishida\'s invention of the control chart, which he called “makimono”, used to control the production process of lightbulbs in the same year. Kanzo Kiribuchi further developed the concept by introducing the term "Hinshitsu Tosei" in his book The Science of Industrial Control, published in 1931. Pearson\'s "The Application of Statistical Methods to Industrial Standardization and Quality Control", which became BS 600, was also published in 1935. Dr. Shewhart\'s Statistical Method from the Viewpoint of Quality Control was published in 1936. \n\nIn 1937, Kaoru Ishikawa entered the Department of Applied Chemistry at the Faculty of Engineering at Tokyo Imperial University. Durin

In [123]:
query = "write Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance"
qa.run(query)

'• Introduction to Inspection, Quality Control, and Quality Assurance: Foundational concepts for quality management.\n\n• Inspection: The process of examining materials and products to verify that they conform to specified requirements. \n• Quality Control: The process of ensuring that products meet or exceed customer expectations. \n• Quality Assurance: A set of policies, procedures, and processes that organizations use to ensure that their products and services meet or exceed customer expectations.\n\n• Chronology of Events:\n- 1946: The Great Japan Society of Technology was dissolved and replaced by the Union of Japanese Scientists and Engineers (JUSE). \n- 1947: The Research Institute for Electrical Communication introduced sampling inspection on purchased products. \n- 1948: The Quality Control Research Group was established and the Quality Control Basic Course was held. \n- 1949: Magil and others instructed QC at NEC’s Tamagawa Plant; the Rice Statistics Mission, including Dr. De

In [124]:
query = "The rise of statistical process control and its significance"
qa.run(query)

"As a quality management consultant, it is important to recognize the rise of statistical process control and its significance. This began in 1931 with the publication of Walter Shewhart’s Economic Control of Quality of Manufactured Products. Shortly thereafter, Yasushi Ishida devised the control chart, which he called “makimono,” and used to control the production process of lightbulbs (1931). Kanzo Kiribuchi introduced quality control as “Hinshitsu Tosei” in his book The Science of Industrial Control (1931). In 1935, Pearson published “The Application of Statistical Methods to Industrial Standardization and Quality Control,” which later became BS 600. Shewhart’s Statistical Method from the Viewpoint of Quality Control was also published in 1936. \n\nThe rise of statistical process control is significant because it allowed for improved quality control and assurance in production processes. This was especially important in post-World War II Japan's industrial comeback, and is still use

## III. The Birth and Evolution of TQM
- Introduction to the concept of TQM
- Overview of the principles and practices of TQM
- The role of TQM in post-World War II Japan's industrial comeback
- Adoption and evolution of TQM in the United States and around the world

In [125]:
query = "The rise of statistical process control and its significance"
qa.run(query)

'As a quality management consultant, I can provide a brief overview of the rise of statistical process control and its significance in the modern business world. This can be seen through the work of several key figures in the industry including Walter Shewhart, Yasushi Ishida, and Kanzo Kiribuchi. \n\nWalter Shewhart published his Economic Control of Quality of Manufactured Products in 1931. This book was the first to provide a framework for statistical process control (SPC) and was a major step in the development of quality control.\n\nYasushi Ishida devised the control chart in 1931, which he referred to as “makimono” and used to control the production process of light bulbs. Kanzo Kiribuchi introduced quality control as “Hinshitsu Tosei” in his book The Science of Industrial Control, also published in 1931.\n\nIn 1935, Pearson published “The Application of Statistical Methods to Industrial Standardization and Quality Control” which later became BS 600. Shewhart’s Statistical Method 

In [126]:
query = "The rise of statistical process control and its significance"
qa.run(query)

'As a quality management consultant, I can provide the following information about the rise of statistical process control and its significance. \n\nStatistical process control (SPC) emerged in the early 20th century as a means of improving the quality of manufactured products. In 1931, Walter Shewhart published “Economic Control of Quality of Manufactured Products”, which laid the groundwork for SPC. Additionally, Yasushi Ishida devised the control chart, which he called “makimono”, and used to control the production process of lightbulbs also in 1931. Kanzo Kiribuchi introduced quality control as “Hinshitsu Tosei” in his book “The Science of Industrial Control” in the same year. \n\nIn 1935, Pearson’s “The Application of Statistical Methods to Industrial Standardization and Quality Control” was published, which later became BS 600. In 1936, Shewhart’s “Statistical Method from the Viewpoint of Quality Control” was published. \n\nIn the 1940s, the Japanese Union of Scientists and Engin

In [127]:
query = "The rise of statistical process control and its significance"
qa.run(query)

'As a quality management consultant, I can provide the following information regarding the rise of statistical process control and its significance:\n\n• The concept of Statistical Process Control (SPC) was first introduced in 1931 by Walter Shewhart’s Economic Control of Quality of Manufactured Products.\n\n• Yasushi Ishida also introduced the control chart, which he called “makimono,” to control the production process of lightbulbs in 1931. \n\n• Kanzo Kiribuchi introduced quality control as “Hinshitsu Tosei” in his book The Science of Industrial Control in the same year.\n\n• In 1935, Pearson’s “The Application of Statistical Methods to Industrial Standardization and Quality Control” was published and it later became BS 600. \n\n• In 1936, Shewhart published Statistical Method from the Viewpoint of Quality Control. \n\n• Statistical process control was further developed and adopted by the Japanese Union of Scientists and Engineers (JUSE) in the 1970s. \n\n• In 1974, JUSE published t

## IV. The Role of Kaoru Ishikawa in the Development of TQM
- Introduction to Ishikawa: his background and career
- Ishikawa's contribution to the development of Quality Circles
- The development and importance of the Ishikawa (cause-and-effect) Diagram
- Ishikawa's philosophical contribution to TQM: the 'next process is your customer'


In [128]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Introduction to Ishikawa: his background and career"
qa.run(query)

'Kaoru Ishikawa played an instrumental role in the development of Total Quality Management (TQM). He was a leading figure in the post-World War II Japanese industrial comeback and a pioneer of quality management. His contributions to the development of TQM included: \n\n• Developing the concept of Quality Circles: In 1965, Ishikawa developed the concept of Quality Circles, which are groups of workers who meet regularly to identify and solve quality-related problems. This concept was adopted by many Japanese companies and has since spread throughout the world.\n\n• Introducing the Ishikawa (cause-and-effect) Diagram: In 1968, Ishikawa introduced the Ishikawa (cause-and-effect) Diagram, which is used to identify the root causes of quality-related problems. This diagram has become an important tool for quality management professionals.\n\n• Philosophical Contribution to TQM: Ishikawa\'s philosophical contribution to TQM was his "next process is your customer" principle, which emphasized t

In [129]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Ishikawa's contribution to the development of Quality Circles"
qa.run(query)

"Kaoru Ishikawa is credited as the father of Total Quality Management (TQM), and his contributions to the development of Quality Circles are widely celebrated and highly influential. Ishikawa's work began in 1965 when he joined the 2nd Quality Control (QC) Overseas Research Team as the head of the delegation (JUSE). During this trip, Ishikawa introduced the concept of Quality Circles to the American Society for Quality Control (ASQC) Annual Conference. \n\nIn 1971, on the occasion of Ishikawa's visit to Taiwan, the first National QC Circle Conference was held (PETT, Taiwan). In addition, Ishikawa became the chair of the ISO/TC 69 Japan Committee, a lecturer on the “Production Control Course for Machinery Manufacturers” for participants from developing countries, and the Vice-President of the Japanese Society for Quality Control (– 74). \n\nIshikawa's most significant contribution to the development of Quality Circles was the publication of his book Control Points of Quality (joint auth

In [130]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - The development and importance of the Ishikawa (cause-and-effect) Diagram"
qa.run(query)

"Kaoru Ishikawa was a Japanese engineer and management theorist who played a significant role in the development of Total Quality Management (TQM). His work and teachings had a major impact on modern business practices, and he is credited with developing the Ishikawa (cause-and-effect) Diagram, also known as the fishbone diagram. \n\nThe Ishikawa diagram is a visual tool used to identify potential causes of a problem in a systematic way. It is a key tool in quality management and process improvement, and it can be used to identify root causes and develop solutions to complex issues. This method was first developed by Ishikawa in 1943 during his work on quality control at the Tokyo Shipyard. \n\nIn the 1950s, Ishikawa further developed the cause and effect diagram and refined the methodology. He wrote a book titled “What is Total Quality Control?”, which outlined his philosophy and the importance of quality control. In this book, he highlighted the importance of understanding the underl

In [131]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Ishikawa's philosophical contribution to TQM: the 'next process is your customer'"
qa.run(query)

"\nKaoru Ishikawa played an instrumental role in the development of Total Quality Management (TQM). He was a major advocate for the concept, and his efforts helped to shape the modern practice of quality management.\n\nIshikawa's contributions to TQM include the development of Quality Circles, the Ishikawa (cause-and-effect) Diagram, and his philosophical concept of 'the next process is your customer'.\n\nQuality Circles:\nIn 1965, Ishikawa published How to Operate Quality Control Circles, which is credited with popularizing the concept of Quality Circles (QC) in Japan. Quality Circles involve small teams of employees who meet regularly to identify and address problems in the workplace. They are an effective way to increase employee engagement and to boost morale.\n\nIshikawa (cause-and-effect) Diagram:\nIn 1976, Ishikawa developed the Ishikawa (cause-and-effect) diagram, a tool for visualizing the potential causes of a problem. The Ishikawa Diagram is widely used in quality management

## V. The Role of the Japanese Union of Scientists and Engineers (JUSE)
- Background of JUSE and its role in the development of quality management in Japan
- Introduction of the Deming Prize by JUSE and its significance
- JUSE's role in promoting TQM and its principles in Japan and globally


In [132]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Background of JUSE and its role in the development of quality management in Japan"
qa.run(query)

'The role of Kaoru Ishikawa in the development of Total Quality Management (TQM) is a key factor in the success of the Japanese Union of Scientists and Engineers (JUSE) in promoting quality management worldwide. The contributions of Kaoru Ishikawa and JUSE to the development of TQM are detailed below. \n\nBackground of JUSE and its Role in the Development of Quality Management in Japan:\n• Kaoru Ishikawa, a professor at the University of Tokyo, founded the Japanese Union of Scientists and Engineers (JUSE) in 1965 with the aim of promoting quality management and research in Japan. \n• JUSE was instrumental in the development of the Quality Control (QC) Circle movement, which was established to encourage the involvement of workers in the improvement of quality and productivity. \n• JUSE also established the General Quality Control Award in 1965, which is awarded annually to individuals and groups who have made outstanding contributions to quality management. \n• JUSE was instrumental in 

In [133]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - ntroduction of the Deming Prize by JUSE and its significance"
qa.run(query)

"Kaoru Ishikawa, a Japanese engineer, was a major contributor to the development of Total Quality Management (TQM). Ishikawa was instrumental in introducing and popularizing the Deming Prize, which is an award given by the Japanese Union of Scientists and Engineers (JUSE) to recognize companies that have achieved excellence in quality management.\n\nThe Deming Prize was established in 1951 and is one of the most prestigious awards in the field of quality management. The award recognizes companies that demonstrate excellence in the implementation of TQM principles, including continuous improvement, customer satisfaction, and employee involvement. It is considered to be one of the most influential quality awards in the world and has been awarded to hundreds of companies from around the world.\n\nIshikawa's involvement in the development of the Deming Prize was significant. He was a member of the committee that established the prize and also helped to draft the criteria for awarding the p

In [134]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - JUSE's role in promoting TQM and its principles in Japan and globally"
qa.run(query)

"Kaoru Ishikawa was an integral part of the development of Total Quality Management (TQM) and its principles in Japan and globally. His work as a professor of engineering at Tokyo University, combined with his roles in the Japanese Union of Scientists and Engineers (JUSE), had a major impact on the development of TQM. \n\nIshikawa's Contributions to TQM: \n• 1965: Joined the 2nd QC Overseas Research Team as the head of the delegation, introduced the activities of QC circles at the ASQC Annual Conference \n• 1971: Awarded with the Grant Medal Deming Application Prize \n• 1972: Joined the 7th QC Overseas Research Team as the head of the delegation and introduced PL issues to Japan \n• 1978: Chair of the ICQCC ’78-TOKYO Program Committee and became a member of the International Cooperation Committee \n• 1979: Chair of the 1st QC Research Delegation Team and visited China \n• 1980: Member of the Nuclear Power Plant Quality Assurance Review Committee \n• Published How to Establish Product S

## VI. The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices
- Analysis of the role TQM plays in today's business and management practices
- The legacy of Ishikawa's methods and tools in current quality management
- Influence of JUSE's principles in contemporary quality initiatives and awards


In [135]:
query = "write about: The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices - Analysis of the role TQM plays in today's business and management practices"
qa.run(query)

"\nToday, Total Quality Management (TQM) is one of the most widely adopted quality management systems in the business world. Its principles and practices have been adopted by companies globally and have been instrumental in improving product and service quality, increasing customer satisfaction, and creating a culture of continuous improvement.\n\nThe development and evolution of TQM can be largely attributed to the contributions of Japanese professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE). Ishikawa's development of Quality Circles, the Ishikawa (cause-and-effect) Diagram, and his 'next process is your customer' philosophy are among the most widely used quality management tools in the world. JUSE's introduction of the Deming Application Prize and its promotion of TQM around the world have also had a significant impact on contemporary business practices. \n\nThe Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices\n\nTQM has had a tremen

In [136]:
query = "write about: The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices - The legacy of Ishikawa's methods and tools in current quality management"
qa.run(query)

"\nThe Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices\n\nThe legacy of Kaoru Ishikawa's methods and tools in current quality management is an important part of modern business practices. His work and that of the Japanese Union of Scientists and Engineers (JUSE) have had a significant impact on the field of quality management. \n\nIshikawa's methods and tools are still widely used today, with many businesses utilizing his Quality Circles and Ishikawa (cause-and-effect) Diagrams to improve their quality control processes. Ishikawa's philosophy of 'next process is your customer' has also had a significant impact on contemporary business practices, with many businesses striving to create a customer-centric approach to their quality management processes. \n\nThe Deming Application Prize, which was established by JUSE in 1980, has also had a major impact on contemporary business practices. This award is now an internationally recognized award that recognizes the efforts

In [137]:
query = "write about: The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices - Influence of JUSE's principles in contemporary quality initiatives and awards"
qa.run(query)

'The impact of Total Quality Management (TQM), Kaoru Ishikawa, and the Japanese Union of Scientists and Engineers (JUSE) on contemporary business practices is evident in the number of initiatives and awards they have created. JUSE has been instrumental in promoting TQM and its principles in Japan and globally, and its influence is reflected in the following initiatives and awards: \n\n• The Deming Application Prize – established by JUSE in 1951, it is the world’s oldest and most prestigious quality award, and is given to companies who demonstrate excellence in quality management. \n\n• The Award for Young Researcher’s Encouragement – established by JUSE in 1981, this award recognizes and encourages young researchers in the field of quality management. \n\n• The Award of the Excellent Factories in Implementation of Industrial Standardization – established by JUSE in 1981, this award recognizes companies that have achieved excellence in the implementation of industrial standardization. \

## VII. Conclusion
- Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management
- Restatement of Thesis
- Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape

In [143]:
query = "write about: Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management"
qa.run(query)

'Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management:\n\nKaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE) played a pivotal role in the development of Total Quality Management (TQM) and the field of quality management. \n\nIshikawa: \n• Contributed to the development of Quality Circles \n• Developed the Ishikawa (cause-and-effect) Diagram \n• Propounded the ‘next process is your customer’ philosophy \n\nJUSE: \n• Established the Deming Application Prize in 1945 \n• Created the Quality Control Seminar series in 1959 \n• Promoted TQM and its principles both in Japan and globally \n• Awarded the Nikkei Quality Control Literature Prize in 1972 \n• Established the Award of the Excellent Factories in Implementation of Industrial Standardization in 1972 \n• Established the International Standardization Activities Cooperation Center in 1974 \n• Invited Dr. Juran to conduct the QC Seminar for Executives in 1971 \n• Establi

In [144]:
query = "Restatement of Thesis"
qa.run(query)

'• Thesis: Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE) have played a critical role in the development and evolution of Total Quality Management (TQM).\n\n• TQM has seen significant changes since its early development in post-World War II Japan, with Kaoru Ishikawa contributing greatly to its advancement through the introduction of Quality Circles and Ishikawa Diagrams.\n\n• JUSE has also been a major force in the promotion of TQM, contributing to its spread around the world and to the establishment of awards such as the Deming Prize.\n\n• TQM, Ishikawa, and JUSE have had a profound impact on modern business practices, with their principles and practices still being applied today in the global business landscape.'

In [145]:
query = "write about: Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape"
qa.run(query)

'\nFuture implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape\n\nThe legacy of TQM, Ishikawa, and JUSE continue to provide invaluable guidance to contemporary business and management practices. By understanding the principles and practices established by these three pioneering organizations, organizations can ensure that their quality management system is effective and efficient. Here are some key points that can be considered to ensure successful future applications of TQM, Ishikawa, and JUSE principles: \n\n• Utilize the Ishikawa (cause-and-effect) Diagram to identify root causes of process issues to ensure that corrective action is taken and that a preventative solution is implemented. \n\n• Leverage the JUSE Deming Prize to recognize organizations that have made significant advances in quality management. \n\n• Align with the principles of TQM such as customer focus, employee involvement, continual improvement, and process orient